# Importar librerias

In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy
import psycopg2

# Punto 1

## Conectar a la BD - crear tabla

In [ ]:

conn_mysql = psycopg2.connect( database="postgres",
                         user='postgres',
                         password='postgres',
                         host='localhost',
                         port='5432')
# you must create a Cursor object. It will let
#  you execute all the queries you need
cur = conn_mysql.cursor()


OperationalError: connection to server at "190.249.20.164", port 5432 failed: Connection timed out
	Is the server running on that host and accepting TCP/IP connections?


In [ ]:
conn_mysql

<connection object at 0x000001BBF5F6DAD0; dsn: 'user=postgres password=xxx dbname=postgres host=localhost port=5432', closed: 0>

In [ ]:

# Use all the SQL you like
cur.execute("""CREATE TABLE TBL_Prueba_MediosPago_Nicolas_Medina (
            column1 numeric,
            column2 numeric,
            column3 numeric,
            cliente varchar,
            column5 numeric,
            fecha numeric,
            medio_pago varchar,
            ventas float,
            column9 varchar)
            """)

cur.close()
conn_mysql.commit()

# Punto 2

# Funcion para insertar datos de un DF a la tabla

In [ ]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras as extras
import pandas as pd


def execute_values(conn, df, table):

    tuples = [tuple(x) for x in df.to_numpy()]

    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()

# leer datos de un TXT y pasarlos a un DF

In [ ]:
df_to_load=pd.read_csv(r"EPIK_MEDIO_20240110.txt",sep='|',header=None,encoding='iso-8859-1')

In [ ]:
df_to_load

,0,1,2,3,4,5,6,7,8
0,1001,40294,1001054029412102023,3-121-546,18694,12102023,Visa-Master Otros,14.96,Retail
1,1001,40295,1001054029512102023,3-90-2473,18694,12102023,Visa-Master Otros,8.86,Retail
2,1001,40296,1001054029612102023,3-710-478,18694,12102023,Debito-OtroBanco,50.00,Retail
3,1001,40297,1001054029712102023,3-728-143,18694,12102023,Debito-OtroBanco,184.56,Retail
4,1001,40301,1001054030112102023,3-710-478,18694,12102023,Debito-OtroBanco,8.59,Retail
...,...,...,...,...,...,...,...,...,...
13293,1010,602,1010230060212102023,8-819-2065,995196,12102023,Gift Card,114.42,Retail
13294,1010,602,1010230060212102023,8-819-2065,995196,12102023,Listo,29.11,Retail
13295,1010,609,1010230060912102023,,995196,12102023,Efectivo,3.19,Retail
13296,1010,616,1010230061612102023,8-786-468,995196,12102023,Debito-OtroBanco,18.18,Retail


In [ ]:
df_to_load.columns=['column1','column2','column3',
            'cliente' ,
            'column5' ,
            'fecha' ,
            'medio_pago' ,
            'ventas' ,
            'column9' ]

In [ ]:
df_to_load

,column1,column2,column3,cliente,column5,fecha,medio_pago,ventas,column9
0,1001,40294,1001054029412102023,3-121-546,18694,12102023,Visa-Master Otros,14.96,Retail
1,1001,40295,1001054029512102023,3-90-2473,18694,12102023,Visa-Master Otros,8.86,Retail
2,1001,40296,1001054029612102023,3-710-478,18694,12102023,Debito-OtroBanco,50.00,Retail
3,1001,40297,1001054029712102023,3-728-143,18694,12102023,Debito-OtroBanco,184.56,Retail
4,1001,40301,1001054030112102023,3-710-478,18694,12102023,Debito-OtroBanco,8.59,Retail
...,...,...,...,...,...,...,...,...,...
13293,1010,602,1010230060212102023,8-819-2065,995196,12102023,Gift Card,114.42,Retail
13294,1010,602,1010230060212102023,8-819-2065,995196,12102023,Listo,29.11,Retail
13295,1010,609,1010230060912102023,,995196,12102023,Efectivo,3.19,Retail
13296,1010,616,1010230061612102023,8-786-468,995196,12102023,Debito-OtroBanco,18.18,Retail


# Cargar datos usando la funcion

In [ ]:
execute_values(conn_mysql, df_to_load, "TBL_Prueba_MediosPago_Nicolas_Medina")

the dataframe is inserted


# Punto 3 - crear vista
(parecido a un SD lo automatizaria con python o datafactory)

In [ ]:
cur = conn_mysql.cursor()
cur.execute("""
            CREATE VIEW view1 AS (
            select cliente,fecha,medio_pago,sum(ventas)
            from TBL_Prueba_MediosPago_Nicolas_Medina
            group by cliente,fecha,medio_pago)

            """)
cur.close()
conn_mysql.commit()

# Punto 4 - Crear Vista

In [ ]:
conn_mysql.rollback()


In [ ]:
cur = conn_mysql.cursor()
cur.execute("""
            CREATE VIEW VW_Prueba_Vacios_NicolasMedinaCaraballo AS (
            select fecha,count(cliente)
            from TBL_Prueba_MediosPago_Nicolas_Medina
            where cliente =' '
            group by fecha)
            """)
cur.close()
conn_mysql.commit()

# Punto 5 - Crear Vista

In [ ]:
cur = conn_mysql.cursor()
cur.execute("""
            CREATE VIEW VW_Prueba_ResumenVentas_NicolasMedina AS (
            select fecha,sum(ventas)
            from
            TBL_Prueba_MediosPago_Nicolas_Medina
            group by fecha)
            """)
cur.close()
conn_mysql.commit()

# Punto 6 - cargar un query a un DF y pasarlo a un txt

In [ ]:
df_to_extract=pd.read_sql('select * from TBL_Prueba_MediosPago_Nicolas_Medina', conn_mysql)

df_to_extract.to_csv("ResumenClientes_Nicolas_Medina_Caraballo.txt",sep=';')

C:\Users\nicki\AppData\Local\Temp\ipykernel_12040\2728333470.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_to_extract=pd.read_sql('select * from TBL_Prueba_MediosPago_Nicolas_Medina', conn_mysql)


# Punto 7 - Usando pivot de pandas

In [ ]:
df_to_extract=pd.read_sql('select * from TBL_Prueba_MediosPago_Nicolas_Medina', conn_mysql)

df_to_extract.pivot_table(index='fecha',columns='medio_pago',values='ventas',aggfunc='sum').to_excel("ResumenVentas_NicolasMedina.xlsx")



C:\Users\nicki\AppData\Local\Temp\ipykernel_12040\3850399425.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_to_extract=pd.read_sql('select * from TBL_Prueba_MediosPago_Nicolas_Medina', conn_mysql)


medio_pago,AHC / Yappy,American Express,Cheque,Club Mercancia,Credit Note Issue,Credit Note Redeem,Debito-OtroBanco,DébitoBac,DébitoBanistmo,Efectivo,...,Listo,NaN,TDC eCommerce,TitanCredit,Vale General Cupon,ValePanama,ValePanama Cupon,Venta Especial,Visa-Master Bac,Visa-Master Otros
fecha,,,,,,,,,,,,,,,,,,,,,
12102023.0,235.9,137.93,1765.27,1843.17,-1998.94,2054.51,41105.33,7439.96,73.78,128904.64,...,2288.9,10.0,2628.25,31505.92,9.84,1108.38,142.48,0.0,7600.69,37716.43


In [ ]:
pd.pivot_table(df_to_extract,index='fecha',columns='medio_pago',values='ventas',aggfunc='sum')

medio_pago,AHC / Yappy,American Express,Cheque,Club Mercancia,Credit Note Issue,Credit Note Redeem,Debito-OtroBanco,DébitoBac,DébitoBanistmo,Efectivo,...,Listo,NaN,TDC eCommerce,TitanCredit,Vale General Cupon,ValePanama,ValePanama Cupon,Venta Especial,Visa-Master Bac,Visa-Master Otros
fecha,,,,,,,,,,,,,,,,,,,,,
12102023.0,235.9,137.93,1765.27,1843.17,-1998.94,2054.51,41105.33,7439.96,73.78,128904.64,...,2288.9,10.0,2628.25,31505.92,9.84,1108.38,142.48,0.0,7600.69,37716.43
